In [1]:
# CONCLUSIONS:
#
# - the more epochs, the better for training accuracy BUT there is a limit, if I put more than certain number it fails, I saw recommended is from 3-5 epochs
# - the mor data, the better. I initially tried with 6-8 rows and the result was really bad, then I tried with a 170 rows dataset and it worked better, at least answering cities and degrees
#   but it still needs to improve accuracy with the degrees because they don't match the 100% of the times
# - the answers improved A LOT changing the content of the training data from: { role-system, role-assistant } to: { role-system, role-user (question), role-assistant }
# - there is validation file parameter, it could be interesting to know how is the correct format to use it


In [2]:
import os, re, math, json, random, time, pickle
from collections import Counter
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
from openai import OpenAI
import httpx
from IPython.display import clear_output

In [3]:
model_name = "gpt-4o-mini"

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

openai = OpenAI(http_client=httpx.Client(verify=False))

In [4]:
test_prompts = [
    "what is the temperature in Tegucigalpa today?",
    "what is the temperature in Yamoussoukro today?",
    "what is the temperature in Rabat today?",
    "what is the temperature in Wellington today?",
    "what is the temperature in New York today?"
]

def ask_for_test_prompts(show_model=False):

    if show_model:
        print(f"using {model_name} model\n")
        
    for q in test_prompts:
        print(f"question: {q}\n")
        a = gpt_fine_tuned(q,show_model)
        print(f"  -answer: {a}\n")

In [5]:
def gpt_fine_tuned(user_prompt, show_model=False):

    system_message = "you are an assistant, just provide quick answers"

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]
    response = openai.chat.completions.create(
        model=model_name, 
        messages=messages,
        seed=42,
        max_tokens=50,
        temperature=0.1
    )
    reply = response.choices[0].message.content
    return reply



In [6]:
# TEST BEFORE FINE TUNING

ask_for_test_prompts(True)

using gpt-4o-mini model

question: what is the temperature in Tegucigalpa today?

  -answer: I don't have real-time data access to provide current temperatures. Please check a weather website or app for the latest information.

question: what is the temperature in Yamoussoukro today?

  -answer: I don't have real-time data access to provide current temperatures. Please check a weather website or app for the latest information.

question: what is the temperature in Rabat today?

  -answer: I don't have real-time data access to provide current temperatures. Please check a weather website or app for the latest information.

question: what is the temperature in Wellington today?

  -answer: I don't have real-time data access to provide current temperatures. Please check a weather website or app for the latest information.

question: what is the temperature in New York today?

  -answer: I don't have real-time data access. Please check a weather website or app for the current temperature in

In [7]:
# load fine tuning data 

with open("./resources/fine_tuning_info.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

with open("./resources/fine_tuning_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [8]:
# run fine tuning

openai.fine_tuning.jobs.create(
    training_file=train_file.id,    
    model="gpt-4o-mini-2024-07-18",
    seed=42,
    hyperparameters={"n_epochs": 1},
    suffix="fine-tuned"
)

FineTuningJob(id='ftjob-ySj7R3UxJuFtEO8QZpqI2CvE', created_at=1748598578, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-l5pA9yLCUC9WzyRoifiw8iGF', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-7Kn48bW5cBi7fiiCA2zivm', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix='fine-tuned', metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None)

In [9]:
openai.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-ySj7R3UxJuFtEO8QZpqI2CvE', created_at=1748598578, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-l5pA9yLCUC9WzyRoifiw8iGF', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-7Kn48bW5cBi7fiiCA2zivm', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix='fine-tuned', metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None)], has_more=True, object='list')

In [10]:
job_id = openai.fine_tuning.jobs.list(limit=1).data[0].id

print(job_id)

ftjob-ySj7R3UxJuFtEO8QZpqI2CvE


In [11]:
openai.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-ySj7R3UxJuFtEO8QZpqI2CvE', created_at=1748598578, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-l5pA9yLCUC9WzyRoifiw8iGF', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-7Kn48bW5cBi7fiiCA2zivm', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix='fine-tuned', metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None)

In [ ]:

finished_job = False

while finished_job == False:
    
    clear_output(wait=True)  # clear output on each iteration

    events = openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data

    for event in events:
        print(f"Event ID: {event.id}")
        print(f"Created At: {event.created_at}")
        print(f"Level: {event.level}")
        print(f"Message: {event.message}")
        print(f"Data: {event.data}")
        print("------")
        
        # flags to exit the loop
        if "The job has successfully completed" in event.message or event.level == "error":
            finished_job = True
    
    time.sleep(20)  # wait 20 seconds on each iteration

Event ID: ftevent-b60ttYSadWzxSxeRORAoaqBT
Created At: 1748599863
Level: info
Message: The job has successfully completed
Data: {}
------
Event ID: ftevent-v5WFd3i20ybnCKgzvn1e2xhx
Created At: 1748599859
Level: info
Message: Usage policy evaluations completed, model is now enabled for sampling
Data: {}
------
Event ID: ftevent-RXVcVeQAenJwrCrnMmL1nnkU
Created At: 1748599068
Level: info
Message: Evaluating model against our usage policies before enabling
Data: {}
------
Event ID: ftevent-RVJjwCw4lugEJ3Xw07uOUg6t
Created At: 1748599068
Level: info
Message: New fine-tuned model created
Data: {}
------
Event ID: ftevent-36H82vxcrz3y6sMRKO5QmKkV
Created At: 1748599037
Level: info
Message: Step 170/170: training loss=0.01
Data: {'step': 170, 'train_loss': 0.011498996056616306, 'total_steps': 170, 'train_mean_token_accuracy': 1.0}
------
Event ID: ftevent-yjkdLjz6oP6I3zU6F8pP6YsH
Created At: 1748599037
Level: info
Message: Step 169/170: training loss=0.00
Data: {'step': 169, 'train_loss': 4.4

In [ ]:
model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
print(fine_tuned_model_name)

In [ ]:
# TEST AFTER FINE TUNING

ask_for_test_prompts(True)